In [55]:
# Импортируем необходимые библиотеки
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression

import pickle

In [56]:
# Импортируем данных
df_nup = pd.read_excel("dataset/X_nup.xlsx")
df_bp = pd.read_excel("dataset/X_bp.xlsx")

In [57]:
# Объединяем два датафрейма функцией merge по столбцу 'Unnamed: 0'. Используем метод Inner.
df_merge = df_bp.merge(df_nup, on = 'Unnamed: 0', how = 'inner')
# Удаляем столбец с нумерацией, который дублирует информацию
df = df_merge.drop(columns = ['Unnamed: 0'], axis = 1)
#Поменяем тип данных для Угла нашивки
df['Угол нашивки, град'] = df['Угол нашивки, град'].astype('float64')

In [58]:
#Переименуем колонки
df.rename(columns = {'Соотношение матрица-наполнитель':'matrix_filler', 
                     'Плотность, кг/м3':'density', 'модуль упругости, ГПа':'elastic_modulus',
                     'Количество отвердителя, м.%':'hardener', 'Содержание эпоксидных групп,%_2':'epoxies', 
                     'Температура вспышки, С_2':'temp', 'Поверхностная плотность, г/м2':'surface_density', 
                     'Модуль упругости при растяжении, ГПа':'tensile_modulus',
                     'Прочность при растяжении, МПа':'tensile_strength', 'Потребление смолы, г/м2':'resin_consumption', 
                     'Угол нашивки, град':'patch_angle', 'Шаг нашивки':'patch_step', 'Плотность нашивки':'patch_density'},
          inplace = True )

In [59]:
df.columns
for i in df.columns:
    perc25=np.quantile(df[i],0.25)
    perc75=np.quantile(df[i],0.75)
    IQR=perc75-perc25
    df.loc[((perc75+1.5*IQR<df[i])|(perc25-1.5*IQR>df[i])),i]=np.median(df[i])
describe = df.describe().T
describe['median'] = df.median()

In [60]:
X = df.copy()
y = X.pop('tensile_strength')
X.shape, y.shape

((1023, 12), (1023,))

In [61]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                                    test_size=0.3, 
                                                    random_state=42)
print('X_train:', X_train.shape, 'y_train:', y_train.shape)
print('X_test:', X_test.shape, 'y_test:', y_test.shape)

X_train: (716, 12) y_train: (716,)
X_test: (307, 12) y_test: (307,)


In [62]:
# Модель линейной регрессии для выборки с целевой переменной 'Прочность при растяжении, ГПа'
model_LinearRegression = LinearRegression()
model_LinearRegression.fit(X_train, y_train)
preds1 = model_LinearRegression.predict(X_test)
preds1[:5]

array([2528.35965871, 2430.2743564 , 2486.8268607 , 2508.08694224,
       2506.98258619])

In [63]:
pickle.dump(model_LinearRegression, open('LR_final.pkl', 'wb'))

In [64]:
model_load = pickle.load(open('LR_final.pkl', 'rb'))

In [65]:
preds2 = model_load.predict(X_test)
preds2[:5]

array([2528.35965871, 2430.2743564 , 2486.8268607 , 2508.08694224,
       2506.98258619])